In [161]:
openai_key = open('../open_ai.key','r').read()

In [162]:
import openai, json
from functools import reduce  # forward compatibility for Python 3
import operator

openai.api_key = openai_key
def completion(new_message_text:str, settings_text:str = '', past_messages:list = []):
    if len(past_messages) == 0 and len(settings_text) != 0:
        past_messages.append({"role": "system", "content": settings_text})
    new_message = {"role": "user", "content": new_message_text}
    past_messages.append(new_message)

    result = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=past_messages,
        max_tokens=100,
    )
    content = result.choices[0].message.content
    response_message = {"role": "assistant", "content": content}
    past_messages.append(response_message)
    response_message_text = content
    return response_message_text, past_messages

In [163]:
import openai

def get_from_dict(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

def set_in_dict(dataDict, mapList, message):
    get_from_dict(dataDict, mapList[:-1])[mapList[-1]] = {'text': message}
    return dataDict

def shunted_chat(chat: dict, messages: dict) -> dict:
    """
    対話APIを使って、対話が分流するようなPythonプログラム
    """
    current = chat
    past_messages = []
    for key in messages['keys']:
        current = current[key]
        past_messages.append(current['text'])
    # print('current:',current)
    past_messages.append({'role':'user','content':messages['text']})
    q = 'Q' + str(len(current.keys()))
    messages['keys'].append(q)
    chat = set_in_dict(chat, messages['keys'], {'role':'user','content':messages['text']})
    current = current[q]
    
    # for m in past_messages:
    #     print(m)
    result = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=past_messages
    )
    # print('res:',json.dumps(result,indent=2))
    messages['keys'].append('A')
    chat = set_in_dict(chat, messages['keys'], result.choices[0]['message'])
    # print(messages['keys'])
    # print(json.dumps(chat,indent=2))

    # messages = [chat[k]['text']['content'] for k in messages['keys'] if ]
    chat_history = []
    current = chat
    for key in messages['keys']:
        chat_history.append(current[key]['text']['role']+': '+current[key]['text']['content'].replace('\n\n\n','\n').replace('\n\n','\n'))
        current = current[key]
    return chat,chat_history




In [164]:
chat = {
    'setting': {'text': {'role': 'assistant', 'content': 'Hello.'}},
    'Q1': {
        'text': {'role': 'user', 'content': 'Hi, how are you?'},
        'A': {
            'text': {'role': 'assistant', 'content': 'I am fine, thank you.'},
        }
    }
}

chat,history = shunted_chat(chat, {'keys': ['Q1', 'A'], 'text': 'What is your name?'})

for m in history:
    print(m)

user: Hi, how are you?
assistant: I am fine, thank you.
user: What is your name?
assistant: I am an AI language model created by OpenAI, so I don't have a name. You can call me OpenAI.


In [165]:
chat = {'setting': {'text': {'role': 'system', 'content': 'あなたは料理研究家です。'}}}
chat,history = shunted_chat(chat, {'keys': [], 'text': 'さつまいもの調理方法を教えてください。'})
print('\n'.join(history))
# print(json.dumps(chat,indent=2,ensure_ascii=False))

user: さつまいもの調理方法を教えてください。
assistant: 
さつまいもの調理方法は、以下のようになります。
1. 皮をむく
さつまいもの皮をピーラーや包丁でむきます。また、皮を残したまま調理することもできます。
2. 切る
必要に応じて、さつまいもを適当な大きさに切ります。例えば、煮物にする場合は一口大に切るとよいでしょう。
3. 煮る
さつまいもを煮物にする場合は、鍋に適量の水を入れて、火にかけます。沸騰したら、さつまいもを加え、中火で20分程度煮ます。柔らかくなったら、火から下ろします。
4. 蒸す
さつまいもを蒸す場合は、蒸し器で15分程度蒸します。
5. 焼く
さつまいもをオーブンで焼く場合は、180℃に予熱したオーブンで30分程度焼きます。また、フライパンでバターやオリーブオイルを加えて焼く方法もあります。
6. 揚げる
さつまいもを天ぷらにする場合は、小麦粉、卵、水、片栗粉を混ぜた衣をつけて、油で揚げます。
以上が、さつまいもの基本的な調理方法となります。


In [166]:
chat,history = shunted_chat(chat, {'keys': [], 'text': 'じゃがいもの調理方法を教えてください。'})
print('\n'.join(history))
print(json.dumps(chat,indent=2,ensure_ascii=False))

user: じゃがいもの調理方法を教えてください。
assistant: 
じゃがいもの調理方法はいくつかありますが、一般的な方法をいくつか紹介します。
1. 煮る
①皮をむき、適当な大きさに切る。
②鍋に水を入れて沸騰させたら、じゃがいもを入れる。
③20〜30分程度火を通す。
④器に取り出し、塩やバターなどで味を調える。
2. 揚げる
①皮をむいて薄くスライスする。
②適量の油をフライパンに熱し、じゃがいもを入れる。
③中火で揚げ焼きにする。
④揚げ色がついたら、キッチンペーパーに取り出して余分な油を吸い取る。
⑤塩やコショウで味を調える。
3. オーブン焼き
①皮をむいたじゃがいもを適当な大きさに切る。
②オリーブオイルやバターなどを使ってじゃがいもをコーティングする。
③オーブンを200度に設定して、焼き始める。
④じゃがいもが柔らかくなり、表面に焦げ目がつくまで焼く。
⑤塩やハーブなどで味を調える。
以上のように、煮る・揚げる・オーブン焼きなど、調理方法によって異なる味わいが楽しめます。楽しみながら試してみてください。
{
  "setting": {
    "text": {
      "role": "system",
      "content": "あなたは料理研究家です。"
    }
  },
  "Q1": {
    "text": {
      "role": "user",
      "content": "さつまいもの調理方法を教えてください。"
    },
    "A": {
      "text": {
        "role": "assistant",
        "content": "\n\nさつまいもの調理方法は、以下のようになります。\n\n1. 皮をむく\nさつまいもの皮をピーラーや包丁でむきます。また、皮を残したまま調理することもできます。\n\n2. 切る\n必要に応じて、さつまいもを適当な大きさに切ります。例えば、煮物にする場合は一口大に切るとよいでしょう。\n\n3. 煮る\nさつまいもを煮物にする場合は、鍋に適量の水を入れて、火にかけます。沸騰したら、さつまいもを加え、中火で20分程度煮ます。柔らかくなったら、火から下ろします。\n\n4. 蒸す\nさつまいもを蒸す場合は、蒸し器で1

In [167]:
chat,history = shunted_chat(chat, {'keys': ['Q2','A'], 'text': 'レシピを一つを教えてください。'})
print('\n'.join(history))

user: じゃがいもの調理方法を教えてください。
assistant: 
じゃがいもの調理方法はいくつかありますが、一般的な方法をいくつか紹介します。
1. 煮る
①皮をむき、適当な大きさに切る。
②鍋に水を入れて沸騰させたら、じゃがいもを入れる。
③20〜30分程度火を通す。
④器に取り出し、塩やバターなどで味を調える。
2. 揚げる
①皮をむいて薄くスライスする。
②適量の油をフライパンに熱し、じゃがいもを入れる。
③中火で揚げ焼きにする。
④揚げ色がついたら、キッチンペーパーに取り出して余分な油を吸い取る。
⑤塩やコショウで味を調える。
3. オーブン焼き
①皮をむいたじゃがいもを適当な大きさに切る。
②オリーブオイルやバターなどを使ってじゃがいもをコーティングする。
③オーブンを200度に設定して、焼き始める。
④じゃがいもが柔らかくなり、表面に焦げ目がつくまで焼く。
⑤塩やハーブなどで味を調える。
以上のように、煮る・揚げる・オーブン焼きなど、調理方法によって異なる味わいが楽しめます。楽しみながら試してみてください。
user: レシピを一つを教えてください。
assistant: 「じゃがバターグラタン」のレシピをお伝えします。
【材料】
・じゃがいも（中サイズ） 2個
・玉ねぎ 1個
・にんにく 1かけ
・ベーコン 4枚
・無塩バター 30g
・小麦粉 大さじ2
・牛乳 300ml
・ナツメグ 少々
・グラタン用チーズ 50g
・パン粉 適量
・塩・こしょう 適量
【作り方】
1.ジャガイモは皮をむき、一口サイズに切ります。玉ねぎ、にんにく、ベーコンも一口サイズに切ります。
2.フライパンにバターを溶かし、玉ねぎ・にんにく・ベーコンを炒めます。
3.小麦粉を加えて炒め、ルーを作ります。
4.牛乳を少しずつ加えながら、ルーを作っていきます。少し塩・こしょう、ナツメグを入れ、味を調えます。
5.グラタン皿に、じゃがいもとルーを入れて、混ぜ合わせます。
6.グラタン用チーズを上にのせ、パン粉を散らします。
7.オーブンを200度に予熱して、グラタンを焼きます。10〜15分焼き、チーズが溶けて金色に焼けたら出来上がりです。
ほかにも、じゃがいもを使ったレシピはたくさんありますので、お好みの料理を作って、美味しい時間を楽しんでください。


In [168]:
def print_chat(d, indent=0):
    for key, value in d.items():
        # print(key,value)
        if isinstance(value, dict):
            if 'text' in value.keys():
                role = value['text']['role']
                content = value['text']['content']
                print(' ' * indent + key + ' ' + role + ': ' + content.replace('\n\n','\n').replace('\n','\n'+' ' * indent))
                value.pop('text')
                print_chat(value, indent+4)
            else:
                print('?',value)
        else:
            print('?',value)
            role = value['role']
            content = value['content']
            print(' ' * indent + key + ' ' + role + ': ' + content)

print_chat(chat)


setting system: あなたは料理研究家です。
Q1 user: さつまいもの調理方法を教えてください。
    A assistant: 
    さつまいもの調理方法は、以下のようになります。
    1. 皮をむく
    さつまいもの皮をピーラーや包丁でむきます。また、皮を残したまま調理することもできます。
    2. 切る
    必要に応じて、さつまいもを適当な大きさに切ります。例えば、煮物にする場合は一口大に切るとよいでしょう。
    3. 煮る
    さつまいもを煮物にする場合は、鍋に適量の水を入れて、火にかけます。沸騰したら、さつまいもを加え、中火で20分程度煮ます。柔らかくなったら、火から下ろします。
    4. 蒸す
    さつまいもを蒸す場合は、蒸し器で15分程度蒸します。
    5. 焼く
    さつまいもをオーブンで焼く場合は、180℃に予熱したオーブンで30分程度焼きます。また、フライパンでバターやオリーブオイルを加えて焼く方法もあります。
    6. 揚げる
    さつまいもを天ぷらにする場合は、小麦粉、卵、水、片栗粉を混ぜた衣をつけて、油で揚げます。
    以上が、さつまいもの基本的な調理方法となります。
Q2 user: じゃがいもの調理方法を教えてください。
    A assistant: 
    じゃがいもの調理方法はいくつかありますが、一般的な方法をいくつか紹介します。
    1. 煮る
    ①皮をむき、適当な大きさに切る。
    ②鍋に水を入れて沸騰させたら、じゃがいもを入れる。
    ③20〜30分程度火を通す。
    ④器に取り出し、塩やバターなどで味を調える。
    2. 揚げる
    ①皮をむいて薄くスライスする。
    ②適量の油をフライパンに熱し、じゃがいもを入れる。
    ③中火で揚げ焼きにする。
    ④揚げ色がついたら、キッチンペーパーに取り出して余分な油を吸い取る。
    ⑤塩やコショウで味を調える。
    3. オーブン焼き
    ①皮をむいたじゃがいもを適当な大きさに切る。
    ②オリーブオイルやバターなどを使ってじゃがいもをコーティングする。
    ③オーブンを200度に設定して、焼き始める。
    ④じゃがいもが柔らかくなり、